In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'february'
new_month = 'march'

In [89]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')
#change call to new filepath
coinmarketcap_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='coinmarketcap',index_col='Ticker')

In [61]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [67]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.loc['VEN'] = df.loc['VET']
df.index = map(lambda x: x.lower(), df.index)

In [68]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Available Supply'] = (d_messari['data']['supply']['circulating'])
    df_all['Supply % Issued'] = df_all['Available Supply'] / (df_all['Y2050 Supply']+0.0001)
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Available Supply']
    df_all['Category'] = (d_messari['data']['misc_data']['categories'][0])
    df_all['Sectors'] = (d_messari['data']['misc_data']['sectors'][0])
    try:
        df_all['Category2'] = (d_messari['data']['misc_data']['categories'][1])
    except:
        pass
    try:
        df_all['Sectors2'] = (d_messari['data']['misc_data']['sectors'][1])
    except:
        pass
    return df_all

In [69]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Available Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        df.loc[t,('Category')] = metrics['Category']
        df.loc[t,('Sectors')] = metrics['Sectors']
        try:
            df.loc[t,('Category2')] = metrics['Category2']
        except:
            pass
        try:
            df.loc[t,('Sectors2')] = metrics['Sectors2']
        except:
            pass
    except:
        pass

In [70]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [71]:
df.loc['IOT'] = df.loc['IOTA']
df.loc['VET'] = df.loc['VEN']

In [72]:
#read in other data sources
onchainfx_db = df

In [90]:
#remove duplicates
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

700
99
144
691
99
144


In [74]:
#fix any ticker mapping issues
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})

In [91]:
onchainfx_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\ocfx_'+new_month+'_2019.xlsx',\
index_col='Ticker')

In [92]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [93]:
#main function to create table of all assets
def create_universe(onchainfx,coinmarketcap,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    for t in eligibility.index:
        if t in coinmarketcap.index:
            eligibility.loc[t,('Volume')] = coinmarketcap.loc[t,('Volume (30d)')]
        else:
            print (t,' no volume data!')
            eligibility.loc[t,('Volume')] = 1
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [94]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [95]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,coinmarketcap_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

In [96]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
Ticker,,,,,,,,,,,,,,,
GRIN,3.899302e+09,1.381707e+07,0.003543,3460620,3.992658,9.766182e+08,0,304981198.0,22.072780,1,0.0,1.0,0.0,1.0,0
HT,5.763176e+08,5.763199e+07,0.100000,50000200,1.152635,5.000000e+08,1,303961237.0,5.274175,1,0.0,3.0,1.0,2.0,0
DCN,5.078808e+08,2.074834e+07,0.040853,326822211298,0.000063,8.000000e+12,0,4278048.0,0.206187,1,1.0,3.0,1.0,2.0,0
QKC,2.986418e+08,2.394062e+07,0.080165,801649919,0.029864,1.000000e+10,0,92364413.0,3.858063,1,1.0,2.0,0.0,2.0,0
BTM,1.842493e+08,8.795703e+07,0.477381,1002499275,0.087738,2.100000e+09,1,84940479.0,0.965704,1,0.0,2.0,0.0,2.0,0
R,1.338552e+08,6.484617e+07,0.484450,484450000,0.133855,1.000000e+09,1,47805965.0,0.737221,1,0.0,3.0,0.0,2.0,0
DENT,9.107246e+07,2.955891e+07,0.324565,32456475076,0.000911,1.000000e+11,1,51044895.0,1.726887,1,0.0,3.0,0.0,2.0,0
NEXO,7.930894e+07,4.441300e+07,0.560000,560000011,0.079309,1.000000e+09,1,119996208.0,2.701826,1,0.0,3.0,1.0,2.0,0
ELA,6.623632e+07,2.938674e+07,0.443665,14640949,2.007161,3.300000e+07,1,41146083.0,1.400158,1,0.0,2.0,0.0,2.0,0


In [97]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [98]:
universe = universe.rename({'HSR' : 'HC'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
Ticker,,,,,,,,,,,,,,,,
BTC,7.962577e+10,6.662026e+10,0.836667,17556200,3794.685887,2.098350e+07,1,1.924724e+11,2.889096,1,1.0,1.0,0.0,3.0,1,1
XRP,3.017641e+10,6.584846e+09,0.218212,21821174210,0.301764,1.000000e+11,1,1.638648e+10,2.488513,1,1.0,2.0,0.0,3.0,1,2
ETH,2.059540e+10,1.471251e+10,0.714359,104993521,140.127803,1.469759e+08,1,9.705392e+10,6.596693,1,1.0,2.0,0.0,3.0,1,3
XLM,1.201126e+10,1.614961e+09,0.134454,19178623630,0.084206,1.426409e+11,1,4.197937e+09,2.599404,1,1.0,1.0,0.0,3.0,1,4
EOS,5.295309e+09,3.286915e+09,0.620722,906245118,3.626961,1.459985e+09,1,2.927942e+10,8.907872,1,1.0,2.0,0.0,3.0,1,5
LTC,3.753514e+09,2.712408e+09,0.722632,60616861,44.746762,8.388348e+07,1,3.052390e+10,11.253430,1,1.0,1.0,0.0,3.0,1,6
BCH,2.776108e+09,2.333790e+09,0.840670,17640138,132.300000,2.098343e+07,1,8.648583e+09,3.705810,1,1.0,1.0,0.0,3.0,1,7
TRX,2.375598e+09,1.584098e+09,0.666821,66682072191,0.023756,1.000000e+11,1,6.263859e+09,3.954211,1,1.0,3.0,1.0,3.0,1,8
ADA,1.948674e+09,1.122743e+09,0.576157,25927070538,0.043304,4.500000e+10,1,6.126591e+08,0.545681,1,1.0,2.0,0.0,3.0,1,9


In [106]:
eligibility_db.loc['FCT']

Marketcap            5.684975e+07
Current Marketcap    5.684975e+07
Float Percent        1.000000e+00
Float                8.745102e+06
Price                6.500753e+00
Total Supply         8.745102e+06
Float Pass           1.000000e+00
Volume               4.945595e+06
Volume Ratio         8.699414e-02
Volume Pass          0.000000e+00
Other Pass           1.000000e+00
Class                2.000000e+00
ERC20                0.000000e+00
Score                2.000000e+00
Eligible             0.000000e+00
Name: FCT, dtype: float64

In [99]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained ADA
Retained BSV
10 full! Do not add  DASH
10 full! Do not add  XMR


In [100]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained MANA
Retained LSK
Retained BTG
Retained ZRX
20 full! Do not add  HOT
20 full! Do not add  WAVES
20 full! Do not add  GAS
20 full! Do not add  SC


In [101]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained STRAT
Retained STORJ
Retained POWR
Retained POLY
Retained DGD
Retained LRC
Retained AION
Retained GNT
Retained PPT
Retained NAS
40 full! No room for WAN
40 full! Do not add  THETA
40 full! Do not add  MCO
40 full! Do not add  WAX
40 full! Do not add  IOTX
40 full! Do not add  ARK
40 full! Do not add  EDO
40 full! Do not add  GXS
40 full! Do not add  CVC
40 full! Do not add  ARDR


In [102]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [103]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [104]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

Application adds []
Application deletes []
Application count members: 24

10 adds []
10 deletes []
10 count members: 10

40 adds ['HOT' 'NULS' 'RVN']
40 deletes ['FCT' 'MONA' 'WAN']
40 count members: 40

Platform adds ['HOT' 'NULS' 'RVN']
Platform deletes ['FCT' 'WAN']
Platform count members: 27

Currency adds []
Currency deletes ['MONA']
Currency count members: 19

20 adds []
20 deletes []
20 count members: 20

Total adds ['HOT' 'NULS' 'RVN']
Total deletes ['FCT' 'MONA' 'WAN']
Total count members: 70

ETH adds []
ETH deletes []
ETH count members: 23

Sector count members: 70


In [107]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])